# 

La idea es usar a `docker compose` como el gran orquestador de servicios que es, por eso vamos a refactorizar todo el codigo, para usar dicho comando desde la raiz del proyecto. Bajo este enfoque, cada servicio va a tener su propio `Dockerfile`, como es el caso de la API, y de APACHE.

```bash
version: '3.8' 

services:
  app-temp:
    build: 
      context: ./my-web-api 👈
      dockerfile: Dockerfile 
    depends_on:
      - tuTiendaDB
    ports:
      - 8000:8000
    volumes:
      - ./my-web-api:/web-server 👈
    networks:
      - platzinet
```

- `context:`: Especifica la ruta al directorio de construcción. Docker Compose buscará el `Dockerfile` dentro de este directorio. Antes `.`

- `volumes`: antes `.:/web-server`

### Añadiendo el servidor Apache como servicio en el DOcker COmpose

Aunque deberia ser un proceso *straight forward*, en realidad pasar de esto:

docker run -d --name my-apache-server --network platzinet -p 80:80 -v `pwd`/web-page:/var/www/html -v `pwd`/apache-config/my-apache-proxy.conf:/etc/apache2/sites-available/000-default.conf ubuntu/apache2 /bin/bash -c "a2enmod proxy proxy_http && apache2ctl -D FOREGROUND"

A configuar el servidor, fue igualmente engorroso. La configuracion en *docker-compose.yml* fue relativamente sencilla con los conocimientos que ya se tienen. Pero toco volver a modificar el *my-apache-proxy.conf*, y forzar a *docker compose* para que volviera *buildiar*, de otra forma, el servicio arrancaba, y se apagaba inmediatemante por un error en la configuracion:


Baja tus servicios y fuerza la reconstrucción del servicio: 

```sh
docker compose down # Baja todos los servicios
docker compose up -d --build apache-server # Construye solo apache-server y luego levanta todo
```



Este es finalmente el *docker-compose.yml*:

```sh
version: '3.8' # Versión de la sintaxis de Docker Compose

services:
  apache-server:
    build:
      context: ./apache-config
      dockerfile: Dockerfile
    ports:
      - 80:80
    volumes:
      - ./web-page:/var/www/html
    networks:
      - platzinet
      
  app-temp:
    build: 
      context: ./my-web-api
      dockerfile: Dockerfile
    depends_on:
      - tuTiendaDB
    ports:
      - 8000:8000
    volumes:
      - ./my-web-api:/web-server
    networks:
      - platzinet

  tuTiendaDB:
    image: mongo:5.0
    ports:
      - 27017:27017
    volumes:
      - dbAppTienda:/data/db
    networks:
      - platzinet
    
    
networks:
  platzinet:
    external: True

volumes: # <--- YOU NEED TO DEFINE THE NAMED VOLUME HERE
  dbAppTienda: # <--- This declares the 'dbAppTienda' volume


```
